In [1]:
# !git clone https://github.com/hssh18121/SegFormer-tf

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
%cd "SegFormer-tf"

/home/work2/Workspace/sonnh/do_an_test/SegFormer-tf


In [3]:
import os
import random
import glob
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from models import SegFormer_B3

2024-05-22 19:34:36.746828: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 19:34:36.877781: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 19:34:36.877808: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 19:34:36.878177: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 19:34:36.937456: I tensorflow/core/platform/cpu_feature_g

In [5]:
# save_train_image_dataset_path = '../save_data_path/train_image_dataset.npy'
# save_val_image_dataset_path = '../save_data_path/val_image_dataset.npy'
# save_train_mask_image_dataset_path = '../save_data_path/mask_train_image_dataset.npy'
# save_val_mask_image_dataset_path = '../save_data_path/mask_val_image_dataset.npy'


In [6]:
# X_train = np.load(save_train_image_dataset_path, mmap_mode='c')
# X_val = np.load(save_val_image_dataset_path, mmap_mode='c')
# y_train = np.load(save_train_mask_image_dataset_path, mmap_mode='c')
# y_val = np.load(save_val_mask_image_dataset_path, mmap_mode='c')

In [7]:
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [8]:
# def _normalize(X_batch, y_batch):
#   X_batch = tf.cast(X_batch, tf.float32)
#   y_batch = tf.cast(y_batch, tf.float32)
#   return X_batch, y_batch

In [9]:
# # Normalize and batch the datasets
# train_dataset = train_dataset.map(_normalize).batch(32)
# val_dataset = val_dataset.batch(32).map(_normalize)

In [4]:
# Paths to the dataset
save_train_image_dataset_path = '../save_data_path/train_image_dataset.npy'
save_val_image_dataset_path = '../save_data_path/val_image_dataset.npy'
save_train_mask_image_dataset_path = '../save_data_path/mask_train_image_dataset.npy'
save_val_mask_image_dataset_path = '../save_data_path/mask_val_image_dataset.npy'

# Load dataset shapes
train_image_shape = np.load(save_train_image_dataset_path, mmap_mode='r').shape
val_image_shape = np.load(save_val_image_dataset_path, mmap_mode='r').shape
train_mask_shape = np.load(save_train_mask_image_dataset_path, mmap_mode='r').shape
val_mask_shape = np.load(save_val_mask_image_dataset_path, mmap_mode='r').shape

# Define generator functions with dtype conversion
def train_generator():
    X_train = np.load(save_train_image_dataset_path, mmap_mode='r')
    y_train = np.load(save_train_mask_image_dataset_path, mmap_mode='r')
    for i in range(train_image_shape[0]):
        yield X_train[i].astype(np.float32), y_train[i].astype(np.float32)

def val_generator():
    X_val = np.load(save_val_image_dataset_path, mmap_mode='r')
    y_val = np.load(save_val_mask_image_dataset_path, mmap_mode='r')
    for i in range(val_image_shape[0]):
        yield X_val[i].astype(np.float32), y_val[i].astype(np.float32)

# Define the output shapes and types
output_signature = (
    tf.TensorSpec(shape=train_image_shape[1:], dtype=tf.float32),
    tf.TensorSpec(shape=train_mask_shape[1:], dtype=tf.float32)
)

# Create TensorFlow datasets from the generators
train_dataset = tf.data.Dataset.from_generator(
    train_generator,
    output_signature=output_signature
)

val_dataset = tf.data.Dataset.from_generator(
    val_generator,
    output_signature=output_signature
)

# Optional: Apply any additional dataset operations (e.g., batching, shuffling)
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

2024-05-22 19:34:41.329541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-22 19:34:41.345054: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-22 19:34:41.345166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
print(train_image_shape[0])

31154


In [6]:
print(train_mask_shape[0])

26154


In [7]:
print(val_mask_shape[0])

5195


In [9]:
print(val_image_shape[0])

5195


In [12]:
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [13]:
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [14]:
checkpoint_path = "../pretrain_weights/augmented_segformer_B3_weights/cp.weights.h5"

In [15]:
model = SegFormer_B3(input_shape = (256,256,3), num_classes = 5)

In [16]:
model.compile('Adam', loss=sm.losses.dice_loss, metrics=[sm.metrics.iou_score],)

In [17]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=30, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                             monitor='val_loss',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             verbose=1)
]

steps_per_epoch = np.floor(train_image_shape[0] / batch_size)

# model = tf.keras.models.load_model('my_model_with_batch_image.keras')
history = model.fit(
   train_dataset,
   steps_per_epoch=steps_per_epoch,
   epochs=200,
   validation_data=val_dataset,
   callbacks=callbacks,
)

Epoch 1/200


2024-05-22 19:01:55.568404: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/seg_former_head/dropout_84/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-05-22 19:01:59.608880: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-05-22 19:01:59.935704: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7470882ac570 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-22 19:01:59.935723: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-05-22 19:01:59.998666: W tensorflow/compiler/xla/service/gpu/buffer_comparator.cc:1052] INTERNAL: ptxas exited with non-zero error code 65280, output: ptxas /tmp/tempfile-ngoc-Z790-D-DDR4-c0484dda-2490940-61909b3aae74c, line 10; fatal   : Unsupporte

 64/973 [>.............................] - ETA: 7:03 - loss: 0.7331 - iou_score: 0.1735

KeyboardInterrupt: 

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
# model.save('diceloss_segformer.keras')